In [ ]:
import pandas as pd 
from selenium.webdriver.common import keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time
from selenium.webdriver.common.by import By  # Import by 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import json 

In [ ]:
s=Service('C:/Users/Sahil Mishra/Downloads/chromedriver.exe')  # location of cromedriver
driver = webdriver.Chrome(service=s)
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}



driver.maximize_window()  # To maximize the window 

while True:
    driver.get("https://www.zomato.com/ncr/gurgaon-restaurants")  # Website to open 

    last_height = driver.execute_script("return document.body.scrollHeight")  # To scroll



    hotellist=[]   # List to store all the data  

    n = 1000

    while n > len(hotellist):
        
        # Scrolling to the end 
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
        time.sleep(2) 
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        last_height = new_height

        hotelURLS=driver.find_elements(By.XPATH, "//div[contains(@id,'root')]//div//a[contains(@href,'order')]")
        time.sleep(2)

        for hotel in hotelURLS[:n]:
            
            # Wait until the element is located
            WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@id,'root')]//div//a[contains(@href,'order')]")))
            
            # Appending all the links into the list hotellist
            hotellist.append(hotel.get_attribute('href'))
            time.sleep(2)
            
    hotellist=[*set(hotellist)]  # To get the unique values 
    time.sleep(10)


Names = []
address = []
numbers = []
ratings = []
city = []


for url in hotellist:
    url = url.replace('https://www.zomato.com','')
    url = 'https://www.zomato.com/webroutes/getPage?page_url='+url+'&location=&isMobile=0'
    time.sleep(5)
    restaurant=requests.get(url, headers=headers)
    restaurant=json.loads(restaurant.text)
    
    # Name of restaurants
    name =  (restaurant["page_data"]["sections"]["SECTION_BASIC_INFO"]["name"])
    Names.append(name)
    time.sleep(2)
    
    # Ratings 
    rating = restaurant["page_data"]["sections"]["SECTION_BASIC_INFO"]["rating"]["aggregate_rating"]
    ratings.append(rating)
    time.sleep(2)
    
    # Phone number 
    num = restaurant["page_data"]["sections"]["SECTION_RES_CONTACT"]["phoneDetails"]["phoneStr"]
    numbers.append(num)
    time.sleep(2)
    
    # address 
    addr = restaurant["page_data"]["sections"]["SECTION_RES_CONTACT"]["address"]
    address.append(addr)
    time.sleep(2)
    
    #city name
    cty = restaurant["page_data"]["sections"]["SECTION_RES_CONTACT"]["city_name"]
    city.append(cty)
    time.sleep(2)
    
# Print all the data 
print(Names)
print(len(Names))
print(address)
print(len(address))
print(numbers)
print(len(numbers))
print(ratings)
print(len(ratings))
print(city)
print(len(city))
print(len(hotellist))

driver.close()

In [ ]:
# Making data frame
col = ["Name", "Address", "Number", "Ratings", "City"]
Restaurants = pd.DataFrame({"Name":Names, "Address":address, "Number": numbers, "Ratings":ratings,"City":city })

In [ ]:
# To check the data 
Restaurants.head()

In [ ]:
# making excel sheet 
Restaurants.to_excel("Restaurants_data2.xlsx")